# Multi-language attacks

TextAttack's four-component framework makes it trivial to run attacks in other languages. In this tutorial, we:

- Create a model wrapper around Transformers [pipelines](https://huggingface.co/transformers/main_classes/pipelines.html) 
- Initialize a pre-trained [CamemBERT](https://camembert-model.fr/) model for sentiment classification
- Load the AlloCiné movie review sentiment classification dataset (from [`datasets`](https://github.com/huggingface/datasets/))
- Load the `pwws` recipe, but use French synonyms from multilingual WordNet (instead of English synonyms)
- Run an adversarial attack on a French language model

Voilà!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_4_CamemBERT.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_4_CamemBERT.ipynb)

In [1]:
from textattack.attack_recipes import PWWSRen2019
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline

import numpy as np

# Quiet TensorFlow.
import os
if "TF_CPP_MIN_LOG_LEVEL" not in os.environ:
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


class HuggingFaceSentimentAnalysisPipelineWrapper(ModelWrapper):
    """ Transformers sentiment analysis pipeline returns a list of responses
        like 
        
            [{'label': 'POSITIVE', 'score': 0.7817379832267761}]
            
        We need to convert that to a format TextAttack understands, like
        
            [[0.218262017, 0.7817379832267761]
    """
    def __init__(self, pipeline):
        self.pipeline = pipeline
    def __call__(self, text_inputs):
        raw_outputs = self.pipeline(text_inputs)
        outputs = []
        for output in raw_outputs:
            score = output['score']
            if output['label'] == 'POSITIVE':
                outputs.append([1-score, score])
            else:
                outputs.append([score, 1-score])
        return np.array(outputs)


In [ ]:
# Create the model: a French sentiment analysis model.
# see https://github.com/TheophileBlard/french-sentiment-analysis-with-bert
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine")
pipeline = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

model_wrapper = HuggingFaceSentimentAnalysisPipelineWrapper(pipeline)

# Create the recipe: PWWS uses a WordNet transformation.
recipe = PWWSRen2019.build(model_wrapper)
#
# WordNet defaults to english. Set the default language to French ('fra')
#
# See "Building a free French wordnet from multilingual resources", 
# E. L. R. A. (ELRA) (ed.), 
# Proceedings of the Sixth International Language Resources and Evaluation (LREC’08).
recipe.transformation.language = 'fra'

dataset = HuggingFaceDataset('allocine', split='test')
for idx, result in enumerate(recipe.attack_dataset(dataset, indices=range(11))):
    print(('x' * 20), f'Result {idx+1}', ('x' * 20))
    print(result.__str__(color_method='ansi'))
    print()


All model checkpoint weights were used when initializing TFCamembertForSequenceClassification.

All the weights of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.
textattack: Unknown if model of class <class '__main__.HuggingFaceSentimentAnalysisPipelineWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Loading nlp dataset allocine, split test.


xxxxxxxxxxxxxxxxxxxx Result 1 xxxxxxxxxxxxxxxxxxxx
Positive (100%) --> Negative (53%)

Magnifique épopée, une belle histoire, touchante avec des acteurs qui interprètent très bien leur rôles (Mel Gibson, Heath Ledger, Jason Isaacs...), le genre de film qui se savoure en famille! :)

bonnard épopée, une beau bobard, touchante avec des acteurs lequel interprètent même acceptablement leur rôles (Mel Gibson, Heath Ledger, Jason Isaacs...), le genre gale pellicule OMS Concepteur savoure un syndicat! :)

xxxxxxxxxxxxxxxxxxxx Result 2 xxxxxxxxxxxxxxxxxxxx
Negative (94%) --> Positive (91%)

Je n'ai pas aimé mais pourtant je lui mets 2 étoiles car l'expérience est louable. Rien de conventionnel ici. Une visite E.T. mais jonchée d'idées /- originales. Le soucis, tout ceci avait-il vraiment sa place dans un film de S.F. tirant sur l'horreur ? Voici un film qui, à l'inverse de tant d'autres qui y ont droit, mériterait peut-être un remake.

Je n'ai pas aimé mais pourtant je lui mets 4 étoiles car l

xxxxxxxxxxxxxxxxxxxx Result 9 xxxxxxxxxxxxxxxxxxxx
Positive (100%) --> Negative (54%)

Un très joli film, qui ressemble à un téléfilm mais qui a le mérite d'être émouvant et proche de ses personnages. Magimel est vraiment très bon et l'histoire est touchante

Un plus joli feuil, qui ressemble à un téléfilm mais qui a le mérite d'être émouvant et proche de ses personnages. Magimel est absolument très lisse et l'histoire est touchante

xxxxxxxxxxxxxxxxxxxx Result 10 xxxxxxxxxxxxxxxxxxxx
Negative (100%) --> Positive (51%)

Mais comment certaines personnes ont pus lui mettre 5/5 et donc dire indirectement que c'est un chef-d'œuvre ??? Et comment a-t-il fait pour sortir au cinéma et non en DTV ??? C'est pas un film que l'on regarde dans une salle obscur ça, pour moi ça ressemble plus à un téléfilm que l'on visionne un dimanche pluvieux pour que les enfants arrête de nous casser les pieds ! Et puis, le scénario avec le chien que devient le meilleur ami du gosse, c'est du vu et revu (un clich